### Riddler Classic

Congratulations, you’ve made it to the fifth round of The Squiddler — a competition that takes place on a remote island. In this round, you are one of the 16 remaining competitors who must cross a bridge made up of 18 pairs of separated glass squares. Here is what the bridge looks like from above:
    
To cross the bridge, you must jump from one pair of squares to the next. However, you must choose one of the two squares in a pair to land on. Within each pair, one square is made of tempered glass, while the other is made of normal glass. If you jump onto tempered glass, all is well, and you can continue on to the next pair of squares. But if you jump onto normal glass, it will break, and you will be eliminated from the competition.

You and your competitors have no knowledge of which square within each pair is made of tempered glass. The only way to figure it out is to take a leap of faith and jump onto a square. Once a pair is revealed — either when someone lands on a tempered square or a normal square — all remaining competitors take notice and will choose the tempered glass when they arrive at that pair.

On average, how many of the 16 competitors will survive and make it to the next round of the competition?


### Sim Approach: Assumptions
- We have participants guess at each step. 
- Every pair has two options, where 0 or 1 represents the tempered glass and must be matched. 


### A Note on Solution:

It was absolutely unecessary for me to have a `dataclass` here, but tryint to play around with some stuff I am learning in `Fluent Python` (which makes it clear to not overuse classes, but hey here we are). 

The nice thing about using a dataclass is I could shuffle contestants and determine which survive each round (attempting to justify the decision). 

There are a lot of slow decisions in the first iteration, this will be cleaned. 

In [3]:
from dataclasses import dataclass
import random 

class Bridge():
    def __init__(self, size = 18):
        self.size = size
        
    def mapTempered(self):
        """Function to randomly determine if tempered is 0 or 1"""
        self.bridge = random.choices([0,1], k=self.size)
        
    def traversal(self, contestants):
        """
        Assuming no shenanigans, it should only take 1 contestant at most to traverse a single step.
        If a contestant falls through, the next contests should avoid the window they fell through
        """
        i = 0 # initial contestant
        for step in self.bridge:
            if contestants[i].choose() != step:
                contestants[i].alive = 0
                i += 1
                if i > 15:
                    return contestants
            else:
                continue
        return contestants
    
@dataclass
class Contestant:
    """class representating a competitor in Squid games"""
    number: int
    alive: int = 1 # they all start out alive

    def choose(self):
        return random.choices([0,1], k=1)[0]

### Run Some Sims: 

We could surely solve this faster in numpy, that is next. 

In [4]:
import time 
sims = [1_000, 10_000, 100_000, 1_000_000, 2_000_000]
for sim in sims:
    start = time.time()
    tot_alive = 0
    for _ in range(sim):
        sequence = [Contestant(i+1) for i in range(16)] # our new sequence -> really unecessary (SLOW)
        b = Bridge(18)
        b.mapTempered()
        sequence = [Contestant(i+1) for i in range(16)]
        output = b.traversal(sequence)
        tot_alive += sum([c.alive for c in output])
    end = time.time()
    print(f"Average survivors for {sim} sims: {tot_alive / sim}")
    print(f"Total time: {end - start:.2f}\n")

Average survivors for 1000 sims: 7.061
Total time: 0.06

Average survivors for 10000 sims: 7.0142
Total time: 0.21

Average survivors for 100000 sims: 6.99966
Total time: 2.01

Average survivors for 1000000 sims: 7.00005
Total time: 20.13

Average survivors for 2000000 sims: 6.998921
Total time: 40.41



### Numpy: 

Still not leveraging numpy in a great way here, but removing a lot of the cruft from the prior simulation. 

In [7]:
import numpy as np

def bridgeGame(size = 18):
    """Simulate a single bridge game in numpy"""
    arr = np.random.randint(2, size=size) 
    deaths = 0
    for i in range(size):
        if random.choice([0, 1]) != arr[i]:
            deaths += 1
    return 16 - deaths

In [8]:
sims = [1_000, 10_000, 100_000, 1_000_000, 2_000_000]
for sim in sims:
    start = time.time()
    tot_alive = 0
    for _ in range(sim):
        tot_alive += bridgeGame(size = 18)
    end = time.time()
    print(f"Average survivors for {sim} sims: {tot_alive / sim}")
    print(f"Total time: {end - start:.2f}\n")

Average survivors for 1000 sims: 6.999
Total time: 0.04

Average survivors for 10000 sims: 7.0031
Total time: 0.14

Average survivors for 100000 sims: 6.99098
Total time: 1.17

Average survivors for 1000000 sims: 6.998791
Total time: 11.60

Average survivors for 2000000 sims: 7.001337
Total time: 23.35



### Analytical Solution

We can think of this as solving for the following:

$1 - Expected Loss$

So really, 1 - (1 * likelihood of 1 death out of 18 + 2 * likelihood of 2 death out of 18....16 * 16 d out of 18 + 16 * 16 d out of 17 + 16 * 16 d out of 16).

Each step can be solved using binomial distribution, and thinking of our "successes" as deaths in this case.
- The probability of observing exactly k deaths in n independent trials, where n is 18 typically
- We then have two additional cases to add...the case when all 16 die at the 17th square, and case when all 16 die at the 16th square. These are both extremelyt rare and push us just over an Expected Loss of 9. 

So we do:

$1 - \sum_{k=1}^{16} k * \binom{n}{k} p^k (1-p)^{n-k} + 16 * \binom{17}{16} p^16 (1-p)^{17-16} + 16 * \binom{16}{16} p^16 (1-p)^{16-16}$

where:
- `k = total deaths`
- `p = likelihood of choosing correct, which is 0.5`
- `n = total squares (usually 18, not always)`

In [8]:
from scipy import stats

def EV(deaths, decisions):
    """Simple function yielding expected value as death * likelihood given binomial distribution"""
    # k is discrete event (typically X), n is total trials - returns EV
    return deaths * stats.binom.pmf(k = deaths, n = decisions, p = 0.5)

# iterate through EVs 1-16 with 18, plus the instances of 17 and 16 (super rare)
expected_deaths = sum([EV(d,18) for d in range(1, 17)]) + EV(deaths = 16, decisions = 17) + EV(deaths = 16, decisions = 16)

print(f"Expected survivors: {16 - expected_deaths}")

Expected survivors: 6.9989166259765625
